In [2]:
#importing libraries
import os
import sys
import warnings

import numpy as np
import pandas as pd
from mizani.formatters import percent_format
from plotnine import *
from datetime import datetime
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy.stats import norm
from IPython.core.display import HTML
from stargazer.stargazer import Stargazer
import statsmodels.nonparametric.kernel_regression as loess

from mizani.transforms import log_trans
from mizani.formatters import percent_format
from mizani.formatters import log_format

warnings.filterwarnings("ignore")

In [3]:
#had to convert 'morg2014.csv' to zip as was too large for github.
file1 = pd.read_csv('ezyzip.zip')

In [5]:
df=pd.DataFrame(file1)
df

,hhid,intmonth,hurespli,minsamp,hrlonglk,hrsample,hrhhid2,serial,hhnum,state,...,ym_file,ym,ch02,ch35,ch613,ch1417,ch05,ihigrdc,docc00,dind02
0,2600310997690,January,1.0,MIS 8,MIS 2-4 Or MIS 6-8 (link To,9100,91001,1,1,63,...,648,633,0,0,0,0,0,18.0,NaN,NaN
1,2600310997690,January,1.0,MIS 8,MIS 2-4 Or MIS 6-8 (link To,9100,91001,1,1,63,...,648,633,0,0,0,0,0,16.0,NaN,NaN
2,2600310997690,January,1.0,MIS 8,MIS 2-4 Or MIS 6-8 (link To,9100,91001,1,1,63,...,648,633,0,0,0,0,0,16.0,Business and financial operations occupations,Administrative and support services
3,41110310970391,January,1.0,MIS 8,MIS 2-4 Or MIS 6-8 (link To,9100,91001,1,1,63,...,648,633,0,0,0,0,0,12.0,NaN,NaN
4,75680310997590,January,1.0,MIS 8,MIS 2-4 Or MIS 6-8 (link To,9100,91001,1,1,63,...,648,633,1,0,1,0,1,14.0,Office and administrative support occupations,"Health care services , except hospitals"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
317051,907086820569600,December,1.0,MIS 8,MIS 2-4 Or MIS 6-8 (link To,9200,92001,1,1,8,...,659,644,0,0,1,0,0,18.0,Management occupations,Public Administration
317052,907086820569600,December,1.0,MIS 8,MIS 2-4 Or MIS 6-8 (link To,9200,92001,1,1,8,...,659,644,0,0,1,0,0,16.0,Management occupations,"Health care services , except hospitals"
317053,947056861569890,December,1.0,MIS 8,MIS 2-4 Or MIS 6-8 (link To,9200,92001,1,1,8,...,659,644,0,0,0,0,0,16.0,NaN,NaN
317054,950868097156649,December,1.0,MIS 8,MIS 2-4 Or MIS 6-8 (link To,9200,92001,1,1,8,...,659,644,0,0,0,0,0,14.0,Office and administrative support occupations,Hospitals
